In [131]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import f1_score
from torch.utils.data import random_split
import torch.nn.functional as F
import os
from PIL import Image
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import wandb

In [148]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # Изменение количества входных каналов на 3
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 3)  # Вывод: 3 класса

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [149]:
class LungDataset(Dataset):
    def __init__(self, image_folder, answers_file, mask_folder=None, transform=None, is_train=True):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.transform = transform
        self.is_train = is_train

        self.image_paths = sorted(os.listdir(image_folder))
        self.mask_paths = sorted(os.listdir(mask_folder)) if mask_folder else None
        
        self.answers_df = pd.read_csv(answers_file)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_folder, self.image_paths[index])
        img = Image.open(img_path)

        if self.mask_folder:
            mask_path = os.path.join(self.mask_folder, self.mask_paths[index])
            mask = Image.open(mask_path)

        if self.transform is not None:
            img = self.transform(img)
            if self.mask_folder:
                mask = self.transform(mask)

        if self.is_train:
            image_id = int((os.path.splitext(self.image_paths[index])[0]).split('_')[1])
            status = self.answers_df.loc[self.answers_df['id'] == image_id, 'target_feature'].values[0]
            return img, mask if self.mask_folder else img, status
        else:
            return img

    def __len__(self):
        return len(self.image_paths)


In [135]:
train_images_folder = "data/train_images"
image_paths = [os.path.join(train_images_folder, filename) for filename in os.listdir(train_images_folder)]
sum_pixels = np.zeros(3)
sum_squared_pixels = np.zeros(3)
count = 0

for image_path in tqdm(image_paths, desc="Нахождение значения Normalize", unit=" image"):
    img = Image.open(image_path).convert("RGB")
    img_array = np.array(img) / 255
    sum_pixels += np.sum(img_array, axis=(0, 1))
    sum_squared_pixels += np.sum(img_array ** 2, axis=(0, 1))
    count += img_array.shape[0] * img_array.shape[1]

mean = sum_pixels / count
std = np.sqrt((sum_squared_pixels / count) - (mean ** 2))
print("Mean:", mean)
print("Std:", std)

Нахождение значения Normalize: 100%|██████████| 27000/27000 [01:35<00:00, 283.99 image/s]

Mean: [0.51394627 0.51394627 0.51394627]
Std: [0.24807256 0.24807256 0.24807256]


In [136]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [137]:
train_images_folder = "data/train_images"
train_masks_folder = "data/train_lung_masks"
answers_file = "data/train_answers.csv"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = LungDataset(image_folder=train_images_folder, mask_folder=train_masks_folder, answers_file=answers_file, transform=transform, is_train=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)

In [138]:
total_size = len(train_dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [139]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [140]:
wandb.init(project='Yandex_ML')

test_f1_score,▁▅▇▇██▇█▄█▇▆▅██
train_loss,█▆▇▅▄▄▄▃▄▃▂▂▂▂▁▂▁▂▁▁▁▁▁▂▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁
test_f1_score,0.88126
train_loss,0.00168


In [141]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [142]:
from sklearn.metrics import f1_score

def calculate_f1_score(model, dataloader, device):
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for data, mask, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    f1_score_value = f1_score(all_targets, all_predictions, average='weighted')
    return f1_score_value

In [143]:
def train_model(model, criterion, optimizer, train_loader, test_loader, epochs, device):
    model.to(device)

    model.train()

    for epoch in range(epochs):
        loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
        for batch_idx, (data, mask, target) in loop:
            data, mask, target = data.to(device), mask.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            loop.set_description(f"Epoch [{epoch + 1}/{epochs}]")
            loop.set_postfix(loss=loss.item())

            # Запись значения функции потерь в W&B
            wandb.log({"train_loss": loss.item()})
        f1_score_value = calculate_f1_score(model, test_loader, device)
        print(f"F1-score on test set: {f1_score_value}")
        
        wandb.log({"test_f1_score": f1_score_value})

    print("Training completed.")

In [144]:
print(next(model.parameters()).device)


cpu


In [145]:
print(torch.cuda.is_available())


True


In [146]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_model(model, criterion, optimizer, train_loader, test_loader, 15, device)

F1-score on test set: 0.809769339634657


F1-score on test set: 0.8329045238370059


F1-score on test set: 0.8395857268488774


F1-score on test set: 0.8711723009115345


F1-score on test set: 0.8757302918542393


...

F1-score on test set: 0.8626379556554593


F1-score on test set: 0.8675977186829846


F1-score on test set: 0.8621467236109785


F1-score on test set: 0.8617615561887939


F1-score on test set: 0.8600279536639484
Training completed.


In [147]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()  
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_image_folder = 'data/test_images'

predictions = []

for filename in os.listdir(test_image_folder):
    img_path = os.path.join(test_image_folder, filename)
    img = Image.open(img_path)
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img)
        _, predicted_class = torch.max(output, 1)
        predictions.append((int(filename.split('.')[0].split('_')[1]), predicted_class.item()))

submission_df = pd.DataFrame(predictions, columns=['id', 'target_feature'])
submission_df.to_csv('submission.csv', index=False)